In [1]:
import pandas as pd

df = pd.read_csv('top_movies_cleaned.csv') # read csv file

Create a DataFrame for the table in SQL with the cast id, name and gender:

In [2]:
df_act = pd.read_csv('API_cast_gender.csv') # read csv file
df_dir = pd.read_csv('API_dir_gender.csv') # read csv file

In [3]:
df_act['person_gender'] = df_act['person_gender'].replace({'Hombre': 'M', 'Mujer': 'F'})

In [4]:
df_act.rename(columns={'person_id': 'actor_id', 'person_name': 'actor_name', 'person_gender': 'actor_gender'}, inplace=True)

In [5]:
df_act.sample(10)

,actor_id,actor_name,actor_gender
1132,1133,George Young,M
2662,2663,Richard Farnsworth,M
737,738,Dave Turner,M
3246,3247,Wendell Corey,M
1358,1359,James Corden,M
1138,1139,Geraldine Viswanathan,F
2268,2269,Michael Madsen,M
2172,2173,Mathilda May,F
56,57,Alan Alda,M
483,484,Carys Bowkett,F


In [6]:
df_act.to_csv('actors.csv', index=False)

Create a DataFrame for the table in SQL with the director id, name and gender:

In [7]:
df_dir['person_gender'] = df_dir['person_gender'].replace({'Hombre': 'M', 'Mujer': 'F'})

In [8]:
df_dir.rename(columns={'person_id': 'director_id', 'person_name': 'director_name', 'person_gender': 'director_gender'}, inplace=True)

In [9]:
df_dir.sample(5)

,director_id,director_name,director_gender
735,736,Robert Stevenson,M
857,858,Tim Miller,M
668,669,Pete Travis,M
756,757,Rupert Sanders,M
209,210,David Yates,M


In [10]:
df_dir.to_csv('directors.csv', index=False)

Create he dataframe to the genres table, with the id and genre name:

In [11]:
import ast
df['genre'] = df['genre'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [12]:
all_genres = [genre for sublist in df['genre'] for genre in sublist]

unique_genres = pd.unique(all_genres)

genres_df = pd.DataFrame(unique_genres, columns=['genre_name'])

C:\Users\almuo\AppData\Local\Temp\ipykernel_17016\3372808026.py:3: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_genres = pd.unique(all_genres)


In [13]:
genres_df['genre_id'] = genres_df.index + 1

In [14]:
genres_df.sample(10)

,genre_name,genre_id
3,Adventure,4
7,Crime,8
9,Action,10
1,Mystery,2
5,Family,6
10,Biography,11
19,Western,20
4,Comedy,5
16,Horror,17
17,Musical,18


In [15]:
genres_df.to_csv('genres.csv', index=False)

Create the dataframe for 'movies'table with all the basic information with 'movie_id', 'title', 'rating', 'meta_score', 'year', 'revenue', 'director_id'

In [16]:
# Make a copy of the dataframe to avoid modifying the original data
movies_df = df[['title', 'rating', 'meta_score', 'year', 'revenue', 'director']].copy()

In [17]:
#Add the movie_id column
movies_df['movie_id'] = movies_df.index + 1

In [18]:
# Translate the director's name to director_id
movies_df = movies_df.merge(df_dir[['director_id', 'director_name']], left_on='director', right_on='director_name', how='left')

In [19]:
#Select only the columns we need
movies_df = movies_df[['movie_id', 'title', 'rating', 'meta_score', 'year', 'revenue', 'director_id']]

In [20]:
movies_df.sample(10)

,movie_id,title,rating,meta_score,year,revenue,director_id
1145,1146,valentines day,5.7,34.0,2010,216485654,270
1497,1498,the dictator,6.4,58.0,2012,179379533,501
895,896,easy a,7.0,72.0,2010,75026327,901
837,838,wonder woman 1984,5.4,60.0,2020,169601036,653
1074,1075,sleepy hollow,7.3,65.0,1999,206071502,856
1804,1805,american made,7.1,65.0,2017,133511855,226
171,172,hunter killer,6.6,43.0,2018,31672678,225
543,544,reservoir dogs,8.3,81.0,1992,2859750,698
116,117,krampus,6.2,49.0,2015,61788393,579
169,170,elemental,7.0,58.0,2023,486797988,685


In [21]:
movies_df.to_csv('movies.csv', index=False)

Create dataframe for cast table with 'movie_id' and 'person_id':

In [22]:
df['cast'] = df['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [23]:
cast_df = df.merge(movies_df[['movie_id', 'title']], how='left', left_on='title', right_on='title')

In [24]:
cast_df = cast_df[['movie_id', 'cast']].explode('cast').rename(columns={'cast': 'actor_name'})

In [25]:
cast_df = cast_df.merge(df_act, how='left', left_on='actor_name', right_on='actor_name')

In [26]:
cast_df = cast_df[['movie_id', 'actor_id']]

In [30]:
cast_df.head(10)

,movie_id,actor_id
0,1,1707
1,1,2081
2,1,1004
3,1,2355
4,2,3120
5,2,1180
6,2,2354
7,2,2520
8,3,972
9,3,2135


In [29]:
cast_df.to_csv('cast.csv', index=False)

Create a dataframe wich relates movies_id and genres_id

In [31]:
df['genre'] = df['genre'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

movie_genres_df = df.merge(movies_df[['movie_id', 'title']], how='left', left_on='title', right_on='title')

movie_genres_df = movie_genres_df[['movie_id', 'genre']].explode('genre').rename(columns={'genre': 'genre_name'})

movie_genres_df = movie_genres_df.merge(genres_df, how='left', left_on='genre_name', right_on='genre_name')

movie_genres_df = movie_genres_df[['movie_id', 'genre_id']]

In [35]:
movie_genres_df.sample(10)

,movie_id,genre_id
2459,905,4
1463,535,5
2001,739,8
2610,963,17
1053,389,12
1426,522,3
1771,652,8
3381,1247,1
512,188,1
4265,1573,11


In [34]:
movie_genres_df.to_csv('movie_genres.csv', index=False)